# Specific Case Study analysis
In this notebook we will produce all the necessary visualizations that are might be useful for the paper.
From these visualizaitons a subset will be used in the paper.

In [1]:
import sys
sys.path.insert(0, '../')

import os
import re
import numpy as np
import networkx as nx
import pyvis.network as network
import seaborn as sns
import pandas as pd
from scipy.stats import mode
import graphviz
import tabulate as tabulate
from graphviz import Digraph
from matplotlib.colors import LinearSegmentedColormap

import matplotlib.pyplot as plt
%matplotlib inline

import tikzplotlib
# Agent Info -> directories, names, ...
from script.experiments_constants import AGENT_EXPERIMENT_INFO
from constants import PROJ_ADDR


# KPI list and kpi effect list 
from script.experiments_constants import ENV_KPI_NAME_LIST
from script.utils import create_effects_list, create_decisions_list, ensure_directory_exists


# Action steering specific loading numerical data and 
from script.load_data import handle_data
from script.symbolic_representation import create_symbolic_state_decision_matrix

# Probabiliry Comparison functions
from script.probability_comparison import plot_and_save_probability_dist_comparison_heatmaps_with_clustering

### Utils
This utility functions will be used as helper to avoid repeating code.

In [3]:
def tikzplotlib_fix_ncols(obj):
    """
    workaround for matplotlib 3.6 renamed legend's _ncol to _ncols, which breaks tikzplotlib
    """
    if hasattr(obj, "_ncols"):
        obj._ncol = obj._ncols
    for child in obj.get_children():
        tikzplotlib_fix_ncols(child)

In [4]:
def create_plot_dir_for_results(analysis_name, result_dir_addrr=proj_address):
    """Create the directory path for storing data."""
    str_helper = f"A1-NetworkSlicingSchedulingPolicy/results/{analysis_name}"
    path = os.path.join(result_dir_addrr, str_helper)
    ensure_directory_exists(path)
    return path

-----

## Plot mean KPI values for all users per slice
These plots will show the mean KPI values for all users both combined and per slice.
The goal is to provide a general overview of the raw numerical data.

In [ ]:
plt.rcParams.update({'font.size': 14})  # Increase the default font size

analysis_name = "Plots_for_Paper"
users = range(3, 7)
agents = ["embb-trf1", "embb-trf2", "urllc-trf1", "urllc-trf2"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0, 1, 2]  # Assuming slices are 0, 1, 2

for agent in agents:
    plot_path = create_plot_dir_for_results(analysis_name + f"/1_Numeric_KPI_Per_Slice/{agent}")
    
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    
    ############# Load all user scenarios data into one dataframe
    agent_numeric_data = pd.DataFrame()
    for number_of_user in users:
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        kpi_data['user_num'] = number_of_user
        agent_numeric_data = pd.concat([agent_numeric_data, kpi_data], axis=0)
        del kpi_data, decision_data
    
    for slice_id in slices:
        # Filter data for the current slice
        slice_data = agent_numeric_data[agent_numeric_data['slice_id'] == slice_id]
        
        # Group by timestep and calculate mean
        mean_kpi_values = slice_data.groupby('timestep')[kpis].mean().reset_index()
        
        # Remove the first 5 timesteps
        mean_kpi_values = mean_kpi_values[mean_kpi_values['timestep'] > 4]
        
        # Create a new figure for each slice
        fig, axes = plt.subplots(3, 1, figsize=(12, 18), sharex=True)
        fig.suptitle(f'Average KPI Values Over Time for {agent} - Slice {slice_id}', fontsize=20)
        
        for i, kpi in enumerate(kpis):
            axes[i].plot(mean_kpi_values['timestep'], mean_kpi_values[kpi])
            axes[i].set_title(f'{kpi}', fontsize=18)
            axes[i].set_ylabel('Average Value', fontsize=16)
            axes[i].tick_params(axis='both', which='major', labelsize=14)
            axes[i].grid(True)
        
        axes[-1].set_xlabel('Timestep', fontsize=16)
        
        plt.tight_layout()
        
        # Save the figure as PNG
        file_name = f"{analysis_name}_{agent}_slice_{slice_id}_kpi_over_time"
        png_file_path = os.path.join(plot_path, file_name + ".png")
        plt.savefig(png_file_path, dpi=300, bbox_inches='tight')
        
        # Save the figure as LaTeX using tikzplotlib
        tex_file_path = os.path.join(plot_path, file_name + ".tex")
        tikzplotlib.save(tex_file_path, figure=fig)
        
        plt.close()

print("All plots have been generated and saved as PNG and LaTeX files.")

##
------

## Plot Probabilistic View of Symbolic Representation
This code blocks will produce the Probability Distributions as one of the probabilistic views of the SymbXRL.

### Prob dist for each agent
This code will create the plot of probability distributions of the agnet's effects on the environment for each agent.
Each plot will show the probability distribution for kpi effects per slice.

In [ ]:
analysis_name = "Plots_for_Paper"
users = range(3, 7)
agents = ["embb-trf1", "embb-trf2", "urllc-trf1", "urllc-trf2"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0, 1, 2]  # Assuming slices are 0, 1, 2

effects_list = create_effects_list()
colors = plt.colormaps['tab10'](np.linspace(0, 1, len(agents)))

for agent in agents:
    plot_path = create_plot_dir_for_results(analysis_name + f"/2_Effect_Probability_Distribution/Effect_Probability_Distribution_per_Agent/{agent}")
    
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    
    ############# Load all user scenarios data into one dataframe
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in users:
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data
    
    # Create a new figure for each agent
    fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)
    fig.suptitle(f'Effect Probability Distribution - {agent}', fontsize=20)
    
    for i, kpi in enumerate(kpis):
        for slice_idx, slice_id in enumerate(slices):
            # Filter data for the current slice
            slice_data = agent_symbolic_data[agent_symbolic_data['slice_id'] == slice_id]
            
            # Calculate the effect probability
            effect_counts = slice_data[kpi].value_counts(normalize=True).reindex(effects_list[kpi], fill_value=0)
            
            # Plot the probability distribution
            axes[i].plot(effect_counts.index, effect_counts.values, marker='o', linestyle='-', 
                         color=colors[slice_id], label=f'Slice {slice_id}')
            axes[i].set_title(f'{kpi}', fontsize=18)
            axes[i].set_xlabel('Effect', fontsize=16)
            axes[i].set_ylabel('Probability' if i == 0 else '', fontsize=16)
            
            # Get the current tick locations and labels
            locs, labels = axes[i].get_xticks(), axes[i].get_xticklabels()
            
            # Set both the locations and the labels
            axes[i].set_xticks(locs)
            axes[i].set_xticklabels(labels, rotation=45, ha='right')
            
            axes[i].tick_params(axis='both', which='major', labelsize=14)
            axes[i].grid(True)
            axes[i].legend(fontsize=12)

    # Adjust the subplot layout to make room for the rotated labels
    plt.tight_layout()
    fig.subplots_adjust(bottom=0.2)
    
    # Save the figure as PNG
    file_name = f"{analysis_name}_{agent}_effect_probability_distribution"
    png_file_path = os.path.join(plot_path, file_name + ".png")
    plt.savefig(png_file_path, dpi=300, bbox_inches='tight')
    
    # Save the figure as LaTeX using tikzplotlib
    tex_file_path = os.path.join(plot_path, file_name + ".tex")
    tikzplotlib_fix_ncols(fig)
    tikzplotlib.save(tex_file_path)
    # plt.show()
    plt.close()
    # break

print("All effect probability distribution plots have been generated and saved as PNG and LaTeX files.")

### Prob dist for each Slice
This code will create the plot of probability distributions of the agnet's effects on the environment for each slice.
In this way we can compare the effect of each agent's effects on each slice of the environment.

In [ ]:
plt.rcParams.update({'font.size': 14})  # Increase the default font size

analysis_name = "Plots_for_Paper"
users = range(3, 7)
agents = ["embb-trf1", "embb-trf2", "urllc-trf1", "urllc-trf2"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0, 1, 2]  # Assuming slices are 0, 1, 2

effects_list = create_effects_list()
colors = plt.colormaps['tab10'](np.linspace(0, 1, len(agents)))

for slice_id in slices:
    plot_path = create_plot_dir_for_results(analysis_name + f"/2_Effect_Probability_Distribution/Effect_Probability_Distribution_per_Slice/Slice_{slice_id}")
    
    # Create a new figure for each slice
    fig, axes = plt.subplots(1, 3, figsize=(24, 8), sharey=True)
    fig.suptitle(f'Effect Probability Distribution - Slice {slice_id}', fontsize=20)
    
    for i, kpi in enumerate(kpis):
        for agent_idx, agent in enumerate(agents):
            agent_info = AGENT_EXPERIMENT_INFO[agent]
            
            ############# Load all user scenarios data into one dataframe
            agent_symbolic_data = pd.DataFrame()
            for number_of_user in users:
                kpi_data, decision_data = handle_data(agent_info, number_of_user)
                symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
                symbolic_effects['user_num'] = number_of_user
                agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
                del symbolic_effects, kpi_data, decision_data
            
            # Filter data for the current slice
            slice_data = agent_symbolic_data[agent_symbolic_data['slice_id'] == slice_id]
            
            # Calculate the effect probability
            effect_counts = slice_data[kpi].value_counts(normalize=True).reindex(effects_list[kpi], fill_value=0)
            
            # Plot the probability distribution
            axes[i].plot(effect_counts.index, effect_counts.values, marker='o', linestyle='-', 
                         color=colors[agent_idx], label=agent)
        
        axes[i].set_title(f'{kpi}', fontsize=18)
        axes[i].set_xlabel('Effect', fontsize=16)
        axes[i].set_ylabel('Probability' if i == 0 else '', fontsize=16)
        
        # Get the current tick locations and labels
        locs, labels = axes[i].get_xticks(), axes[i].get_xticklabels()
        
        # Set both the locations and the labels
        axes[i].set_xticks(locs)
        axes[i].set_xticklabels(labels, rotation=45, ha='right')
        
        axes[i].tick_params(axis='both', which='major', labelsize=14)
        axes[i].grid(True)
        axes[i].legend(fontsize=12)

    # Adjust the subplot layout to make room for the rotated labels
    plt.tight_layout()
    fig.subplots_adjust(bottom=0.2)
    
    
    # Save the figure as PNG
    file_name = f"{analysis_name}_slice_{slice_id}_effect_probability_distribution"
    png_file_path = os.path.join(plot_path, file_name + ".png")
    plt.savefig(png_file_path, dpi=300, bbox_inches='tight')
    
    # Save the figure as LaTeX using tikzplotlib
    tex_file_path = os.path.join(plot_path, file_name + ".tex")
    tikzplotlib_fix_ncols(fig)
    tikzplotlib.save(tex_file_path)
    
    plt.close()

print("All effect probability distribution plots have been generated and saved as PNG and LaTeX files.")

### Prob dist for each KPI
This code will create the plot of probability distributions of the agnet's effects on the environment for each KPI.
In this way we can compare the effect of each agent's effects on each KPI of the environment.

In [ ]:
plt.rcParams.update({'font.size': 14})  # Increase the default font size

analysis_name = "Plots_for_Paper"
users = range(3, 7)
agents = ["embb-trf1", "embb-trf2", "urllc-trf1", "urllc-trf2"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0, 1, 2]  # Assuming slices are 0, 1, 2

effects_list = create_effects_list()
colors = plt.colormaps['tab10'](np.linspace(0, 1, len(agents)))

# Prepare data for all agents
all_agent_data = {}
for agent in agents:
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in users:
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
    all_agent_data[agent] = agent_symbolic_data

for kpi in kpis:
    plot_path = create_plot_dir_for_results(analysis_name + f"/2_Effect_Probability_Distribution/Effect_Probability_Distribution_per_KPI/{kpi}")
    
    # Create a new figure for each KPI
    fig, axes = plt.subplots(1, 3, figsize=(24, 8), sharey=True)
    fig.suptitle(f'Effect Probability Distribution - {kpi}', fontsize=20)
    
    for i, slice_id in enumerate(slices):
        for agent_idx, agent in enumerate(agents):
            # Filter data for the current slice
            slice_data = all_agent_data[agent][all_agent_data[agent]['slice_id'] == slice_id]
            
            # Calculate the effect probability
            effect_counts = slice_data[kpi].value_counts(normalize=True).reindex(effects_list[kpi], fill_value=0)
            
            # Plot the probability distribution
            axes[i].plot(effect_counts.index, effect_counts.values, marker='o', linestyle='-', 
                         color=colors[agent_idx], label=agent)
        
        axes[i].set_title(f'Slice {slice_id}', fontsize=18)
        axes[i].set_xlabel('Effect', fontsize=16)
        axes[i].set_ylabel('Probability' if i == 0 else '', fontsize=16)
        
        # Get the current tick locations and labels
        locs, labels = axes[i].get_xticks(), axes[i].get_xticklabels()
        
        # Set both the locations and the labels
        axes[i].set_xticks(locs)
        axes[i].set_xticklabels(labels, rotation=45, ha='right')
        
        axes[i].tick_params(axis='both', which='major', labelsize=14)
        axes[i].grid(True)
        axes[i].legend(fontsize=12)

    # Adjust the subplot layout to make room for the rotated labels
    plt.tight_layout()
    fig.subplots_adjust(bottom=0.2)
    
    # Save the figure as PNG
    file_name = f"{analysis_name}_{kpi}_effect_probability_distribution"
    png_file_path = os.path.join(plot_path, file_name + ".png")
    plt.savefig(png_file_path, dpi=300, bbox_inches='tight')
    
    # Save the figure as LaTeX using tikzplotlib
    tex_file_path = os.path.join(plot_path, file_name + ".tex")
    tikzplotlib_fix_ncols(fig)
    tikzplotlib.save(tex_file_path)
    
    plt.close()

print("All effect probability distribution plots per KPI have been generated and saved as PNG and LaTeX files.")

##
------

## Graph Plotter Util
This functino will be used to create the pyvis object that will represent the Graph Analysis of the SymbXRL

In [8]:
def plot_graph_from_data_for_presentation(df, column_name):
    """
    This function will receive the data and the column name of the data that you want to plot the graph and 
    returns the networkX object and the pyvis object and the groups color mapping of the nodes
    """
    
    decision_df = []
    # Calculate the Cross Matrix
    cross_data = pd.crosstab(df[column_name], df[column_name].shift(-1), normalize='index')*100
    # Create the directed graph from the transition probabilities
    for i, row in cross_data.iterrows():
        for j, prob in row.items():
            if prob > 0: 
                decision_df.append({
                    'source': i,
                    'target': j,
                    'weight': prob / 20,
                    'size_weight': 1 / prob,
                    'title': f'Percentage: {prob}' ,
                    'width': prob / 5  # Adjust this value to control the edge width
                })

    G = nx.from_pandas_edgelist(pd.DataFrame(decision_df), 'source', 'target', edge_attr=['weight', 'title', 'width'], create_using=nx.DiGraph())
    
    # Add a title attribute to each node containing its name
    for node in G.nodes():
        G.nodes[node]['title'] = node
    
    # Calculate the frequency of each decision in the original DataFrame
    decision_counts = df[column_name].value_counts()
    
    # Apply exponential scaling for node sizes
    min_size = 5
    scale_factor = 0.55  # Adjust this value to control the exponential growth rate of the sizes
    decision_size = {decision: min_size + np.exp(scale_factor * np.log1p(count - 1)) for decision, count in decision_counts.items()}
    
    # Set the size attribute for each node based on the decision count
    for node in G.nodes():
        G.nodes[node]['size'] = decision_size.get(node, min_size)  # Use default size if node not in decision_size

    # Add centrality attributes to the nodes --> Not sure what they do
    nx.set_node_attributes(G, nx.betweenness_centrality(G, weight='size_weight'), 'betweenness_centrality')
    nx.set_node_attributes(G, nx.degree_centrality(G), 'degree_centrality')
    nx.set_node_attributes(G, nx.closeness_centrality(G, distance='size_weight'), 'closeness_centrality')

    # Create the Pyvis network and save it to a file
    net = network.Network(
        height="1500px", 
        width="100%", 
        bgcolor="white",  # Set background color to white
        font_color="black",  # Set default font color to black for visibility
        directed=True, 
        notebook=True, 
        filter_menu=True,  
        select_menu=True, 
        cdn_resources="in_line"
    )
    net.from_nx(G)
    
    decision_counts = df[column_name].value_counts(normalize=1)
    for node in net.nodes:
        node['title'] = f"node name: {node['title']} \n prob: {round(100 * decision_counts[node['id']], 1)}%"
        node['color'] = '#1f78b4'  # Set node color to a visible color, e.g., blue
        node['font'] = {
            'color': 'black',  # Set font color to black
            'size': 50  # Increase the font size
        }
    
    for edge in net.edges:
        edge['width'] = edge['width']  # Set the edge width from the attribute

    # Calculate the graph size
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()

    # Create a text element for the graph size information
    size_text = f"Number of Nodes: {num_nodes}<br>Number of Edges: {num_edges}"

    # Add the size information as an HTML element to the Pyvis network
    net.add_node("size_info", label=size_text, shape="text", x='-95%', y=0, physics=False)

    net.barnes_hut(overlap=1)
    net.show_buttons(filter_=['physics'])

    return G, net, size_text

## Plot of graph of decision Per Agent
In this part we will produce the Graph Analysis results of SymbXRL for each agnet's mode and traffic profile.

### pyvis

This code block will use pyvis to create the graph of the decision making process for each agent and visualize it (will create a html file that is better for playing around).


In [9]:
plt.rcParams.update({'font.size': 14})  # Increase the default font size

analysis_name = "Plots_for_Paper"

users = range(3, 7)
agents = ["embb-trf1", "embb-trf2", "urllc-trf1", "urllc-trf2"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0, 1, 2]  # Assuming slices are 0, 1, 2

effects_list = create_effects_list()
colors = plt.colormaps['tab10'](np.linspace(0, 1, len(agents)))


for agent in agents:
    plot_path = create_plot_dir_for_results(analysis_name + f"/3_Knowledge_Graphs/Graph_of_Decisions_Per_Agent_Pyvis/{agent}")
    
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    
    ############# Load all user scenarios data into one dataframe
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in users:
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        symbolic_effects['user_num'] = number_of_user
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data
    
    # display(agent_symbolic_data)
    # break

    for i, slice_id in enumerate(slices):
        # Filter the data for the current slice and KPI
        slice_data = agent_symbolic_data[agent_symbolic_data['slice_id'] == slice_id].copy()
        
        # Load graph and pyvis object plus group community 
        G, net, _ = plot_graph_from_data_for_presentation(slice_data, 'combined_decision')
        
        file_name = f"Decition_Graph_{agent}_slice-{slice_id}.html"       
        
        full_file_path = os.path.join(plot_path, file_name)
        net.save_graph(full_file_path)
        
        

### Graph viz

This code block will use Graphviz to create the graph of the decision making process for each agent and visualize it (will create a pdf file that is better for looking and using in the paper).

In [ ]:
def plot_graph_from_data_for_paper(df, column_name, output_path):
    cross_data = pd.crosstab(df[column_name], df[column_name].shift(-1), normalize='index')*100
    decision_counts = df[column_name].value_counts(normalize=True)
    
    dot = Digraph(comment='Decision Graph', engine='dot')
    dot.attr(rankdir='LR', size='12,8', dpi='300', bgcolor='white')
    
    # Add nodes
    for node, freq in decision_counts.items():
        node_size = 1 + 2 * freq  # Moderate size difference
        label = node.split(' - ')
        label.append(f'prob: {freq:.1%}')
        label = '\n'.join(label)
        dot.node(node, label, shape='ellipse', 
                 width=str(node_size), height=str(node_size),
                 style='filled', fillcolor='#E6F3FF', color='#4A6FE3',
                 fontname='Arial', fontsize='10')
    
    # Add edges
    for i, row in cross_data.iterrows():
        for j, prob in row.items():
            if prob > 0:
                penwidth = 0.5 + prob/50  # Reduced edge width scaling
                dot.edge(i, j, 
                         label=f'{prob:.1f}%', 
                         penwidth=str(penwidth),
                         color='#4A6FE3',
                         fontname='Arial', fontsize='8',
                         fontcolor='#4A6FE3')
    
    # Add legend
    dot.attr(label='Node size: state frequency | Edge width: transition probability', 
             fontname='Arial', fontsize='12', labelloc='t')
    
    # Save the graph
    dot.render(output_path, format='pdf', cleanup=True)
    
    return dot

plt.rcParams.update({'font.size': 14})  # Increase the default font size

analysis_name = "Plots_for_Paper"

users = range(3, 7)
agents = ["embb-trf1", "embb-trf2", "urllc-trf1", "urllc-trf2"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0, 1, 2]  # Assuming slices are 0, 1, 2

effects_list = create_effects_list()
colors = plt.colormaps['tab10'](np.linspace(0, 1, len(agents)))

for agent in agents:
    plot_path = create_plot_dir_for_results(analysis_name + f"/3_Knowledge_Graphs/Graph_of_Decisions_Per_Agent_Graphviz/Final/{agent}")
    
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    
    ############# Load all user scenarios data into one dataframe
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in users:
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        symbolic_effects['user_num'] = number_of_user
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data
        
    for i, slice_id in enumerate(slices):
        # Filter the data for the current slice and KPI
        slice_data = agent_symbolic_data[agent_symbolic_data['slice_id'] == slice_id].copy()
        
        # Generate and save the graph
        file_name = f"Knowledge_Graph_{agent}_slice-{slice_id}"
        full_file_path = os.path.join(plot_path, file_name)
        
        dot = plot_graph_from_data_for_paper(slice_data, 'combined_decision', full_file_path)
    # break
print("All decision graphs have been generated and saved as PDF files.")

##
------

## Plot Density Maps

THis code block will create the Correlation density map of the agetns' decisions and their effects on the environment. 

### Heatmap of Decision Effect Per Slice for Agent
This code will produce hte density maps of the decision and their effect for each agent. the result will be pdfs that contain the density maps for all slices in one plot.

In [ ]:
plt.rcParams.update({'font.size': 8})  # Reduce default font size for better fit

def plot_heatmaps_for_paper(agent_symbolic_data, agent, effects_list, kpis, slices, output_path):
    # Get all unique decisions across all slices
    all_decisions = agent_symbolic_data['combined_decision'].unique()
    decision_counts = agent_symbolic_data['combined_decision'].value_counts()
    sorted_unique_decisions = decision_counts.index.tolist()

    # Calculate the height based on the number of decisions
    decision_height = 0.3  # Height per decision in inches
    min_height = 8  # Minimum height of the plot
    calculated_height = max(min_height, len(sorted_unique_decisions) * decision_height)

    # Set the figure size with calculated height
    fig, axes = plt.subplots(1, 3, figsize=(28, calculated_height), sharey=True)
    fig.suptitle(f'Effect Probabilities Heatmaps for Agent {agent}', fontsize=16)

    cmap = 'viridis'

    # Concatenate all effects
    all_effects = [effect for kpi in kpis for effect in effects_list[kpi]]

    def format_value(val):
        if val >= 0.001:
            return f'{val*100:.1f}%'  # Show as percentage with one decimal place
        return ''

    for s, slice_id in enumerate(slices):
        slice_data = agent_symbolic_data[agent_symbolic_data['slice_id'] == slice_id]
        
        # Initialize the matrix for all KPIs and all decisions
        div_matrix = np.zeros((len(sorted_unique_decisions), len(all_effects)))

        for i, decision in enumerate(sorted_unique_decisions):
            decision_data = slice_data[slice_data['combined_decision'] == decision]
            
            if not decision_data.empty:
                col_index = 0
                for kpi in kpis:
                    kpi_effects = effects_list[kpi]
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[i, col_index:col_index+len(kpi_effects)] = effect_counts.values
                    col_index += len(kpi_effects)

        # Normalize the matrix for this slice
        div_matrix = div_matrix / np.sum(div_matrix)

        # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
        total_sum = np.sum(div_matrix)
        if not np.isclose(total_sum, 1.0, atol=1e-6):
            print(f"Warning: Sum for slice {slice_id} is {total_sum}, which is not 1.")

        # Create a custom annotation array
        annot = np.vectorize(format_value)(div_matrix)

        # Plot heatmap
        sns.heatmap(div_matrix, ax=axes[s], cmap=cmap, vmin=0, vmax=1, cbar=(s == 2),
                    annot=annot, fmt='', annot_kws={'size': 3})
        
        axes[s].set_title(f'Slice {slice_id}', fontsize=14)
        axes[s].set_xlabel('Effects', fontsize=12)
        if s == 0:
            axes[s].set_ylabel('Decisions', fontsize=12)
        
        # Adjust x-axis ticks with rotated labels
        axes[s].set_xticks(np.arange(len(all_effects)) + 0.5)
        axes[s].set_xticklabels(all_effects, rotation=45, ha='right', va='top', fontsize=6)
        
        # Adjust y-axis ticks
        axes[s].set_yticks(np.arange(len(sorted_unique_decisions)) + 0.5)
        axes[s].set_yticklabels(sorted_unique_decisions, rotation=0, fontsize=8)
    
    # Adjust layout
    plt.tight_layout()
    
    # Save the figure as PDF
    plt.savefig(output_path, dpi=600, bbox_inches='tight')
    
    # Save the figure as LaTeX using tikzplotlib
    tex_output_path = output_path.replace('.pdf', '.tex')
    tikzplotlib.save(tex_output_path)
    
    plt.close()

# Main execution
analysis_name = "Plots_for_Paper"
users = range(3, 7)
agents = ["embb-trf1", "embb-trf2", "urllc-trf1", "urllc-trf2"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0, 1, 2]
effects_list = create_effects_list()

for agent in agents:
    plot_path = create_plot_dir_for_results(analysis_name + f"/4_Heatmaps/Heatmap_of_Decision_Effect_All_Slices_in_one/{agent}")
    
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    
    # Load all user scenarios data into one dataframe
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in users:
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data

    # Generate and save the heatmap
    output_file = os.path.join(plot_path, f"{agent}_heatmaps.pdf")
    plot_heatmaps_for_paper(agent_symbolic_data, agent, effects_list, kpis, slices, output_file)

print("All heatmaps have been generated and saved as PDF files.")

### Heatmap of Decision Effect Per Agent for Slice Seperatly
This code will produce hte density maps of the decision and their effect for each slice. the result will be pdfs that contain the density maps for each slice seperatly.

In [ ]:
plt.rcParams.update({'font.size': 8})  # Reduce default font size for better fit

def plot_heatmaps_for_paper(agent_symbolic_data, agent, effects_list, kpis, slices, output_path):
    # Get all unique decisions across all slices
    all_decisions = agent_symbolic_data['combined_decision'].unique()
    decision_counts = agent_symbolic_data['combined_decision'].value_counts()
    sorted_unique_decisions = decision_counts.index.tolist()

    # Calculate the height based on the number of decisions
    decision_height = 0.3  # Height per decision in inches
    min_height = 8  # Minimum height of the plot
    calculated_height = max(min_height, len(sorted_unique_decisions) * decision_height)

    # Concatenate all effects
    all_effects = [effect for kpi in kpis for effect in effects_list[kpi]]

    cmap = 'viridis'

    for s, slice_id in enumerate(slices):
        # Create a new figure for each slice
        fig, ax = plt.subplots(figsize=(20, calculated_height))  # Increased width to accommodate annotations
        fig.suptitle(f'Effect Probabilities Heatmap for Agent {agent} - Slice {slice_id}', fontsize=16)

        slice_data = agent_symbolic_data[agent_symbolic_data['slice_id'] == slice_id]
        
        # Initialize the matrix for all KPIs and all decisions
        div_matrix = np.zeros((len(sorted_unique_decisions), len(all_effects)))

        for i, decision in enumerate(sorted_unique_decisions):
            decision_data = slice_data[slice_data['combined_decision'] == decision]
            
            if not decision_data.empty:
                col_index = 0
                for kpi in kpis:
                    kpi_effects = effects_list[kpi]
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[i, col_index:col_index+len(kpi_effects)] = effect_counts.values
                    col_index += len(kpi_effects)

        # Normalize the matrix for this slice
        div_matrix = div_matrix / np.sum(div_matrix)

        # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
        total_sum = np.sum(div_matrix)
        if not np.isclose(total_sum, 1.0, atol=1e-6):
            print(f"Warning: Sum for slice {slice_id} is {total_sum}, which is not 1.")

        # Create a custom annotation array
        def format_value(val):
            if val >= 0.001:
                return f'{val*100:.1f}%'  # Show as percentage with one decimal place
            return ''

        annot = np.vectorize(format_value)(div_matrix)

        # Plot heatmap with fixed scale from 0 to 1 and display formatted values
        sns.heatmap(div_matrix, ax=ax, cmap=cmap, vmin=0, vmax=1, cbar=True, 
                    annot=annot, fmt='', annot_kws={'size': 6})
        
        ax.set_title(f'Slice {slice_id}', fontsize=14)
        ax.set_xlabel('Effects', fontsize=12)
        ax.set_ylabel('Decisions', fontsize=12)
        
        # Adjust x-axis ticks with rotated labels
        ax.set_xticks(np.arange(len(all_effects)) + 0.5)
        ax.set_xticklabels(all_effects, rotation=45, ha='right', va='top', fontsize=6)
        
        # Adjust y-axis ticks
        ax.set_yticks(np.arange(len(sorted_unique_decisions)) + 0.5)
        ax.set_yticklabels(sorted_unique_decisions, rotation=0, fontsize=8)

        # Adjust layout
        plt.tight_layout()
        
        # Save the figure as PDF
        slice_output_path = output_path.replace('.pdf', f'_slice_{slice_id}.pdf')
        plt.savefig(slice_output_path, dpi=600, bbox_inches='tight')
        
        # Save the figure as LaTeX using tikzplotlib
        tex_output_path = slice_output_path.replace('.pdf', '.tex')
        tikzplotlib.save(tex_output_path)
        
        plt.close()

# Main execution
analysis_name = "Plots_for_Paper"
users = range(3, 7)
agents = ["embb-trf1", "embb-trf2", "urllc-trf1", "urllc-trf2"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0, 1, 2]
effects_list = create_effects_list()

for agent in agents:
    plot_path = create_plot_dir_for_results(analysis_name + f"/4_Heatmaps/Heatmap_of_Decision_Effect_per_slice/{agent}")
    
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    
    # Load all user scenarios data into one dataframe
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in users:
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data

    # Generate and save the heatmap
    output_file = os.path.join(plot_path, f"{agent}_heatmaps.pdf")
    plot_heatmaps_for_paper(agent_symbolic_data, agent, effects_list, kpis, slices, output_file)

print("All heatmaps have been generated and saved as PDF files.")

### Heatmap of Decision Effect Per Agent for Slice Seperatly cleaned

This code will produce the same density maps as the previous code but will clean the plot to make it more readable.
To make it more readable we will not show the actions that have been made by agent less than 0.01% of the time.

In [ ]:
plt.rcParams.update({'font.size': 8})  # Reduce default font size for better fit

def plot_heatmaps_for_paper(agent_symbolic_data, agent, effects_list, kpis, slices, output_path):
    # Concatenate all effects
    all_effects = [effect for kpi in kpis for effect in effects_list[kpi]]

    cmap = 'viridis'

    def format_value(val):
        if val >= 0.001:
            return f'{val*100:.1f}%'  # Show as percentage with one decimal place
        return ''

    for slice_id in slices:
        slice_data = agent_symbolic_data[agent_symbolic_data['slice_id'] == slice_id]
        
        # Get unique decisions for this slice
        slice_decisions = slice_data['combined_decision'].unique()
        decision_counts = slice_data['combined_decision'].value_counts()
        sorted_slice_decisions = decision_counts.index.tolist()

        # Calculate the height based on the number of decisions in this slice
        decision_height = 0.3  # Height per decision in inches
        min_height = 5  # Minimum height of the plot
        calculated_height = max(min_height, len(sorted_slice_decisions) * decision_height)

        # Create a new figure for each slice
        fig, ax = plt.subplots(figsize=(20, calculated_height))
        fig.suptitle(f'Effect Probabilities Heatmap for Agent {agent} - Slice {slice_id}', fontsize=16)

        # Initialize the matrix for all KPIs and decisions in this slice
        div_matrix = np.zeros((len(sorted_slice_decisions), len(all_effects)))

        for i, decision in enumerate(sorted_slice_decisions):
            decision_data = slice_data[slice_data['combined_decision'] == decision]
            
            if not decision_data.empty:
                col_index = 0
                for kpi in kpis:
                    kpi_effects = effects_list[kpi]
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[i, col_index:col_index+len(kpi_effects)] = effect_counts.values
                    col_index += len(kpi_effects)

        # Normalize the matrix for this slice
        div_matrix = div_matrix / np.sum(div_matrix)

        # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
        total_sum = np.sum(div_matrix)
        if not np.isclose(total_sum, 1.0, atol=1e-6):
            print(f"Warning: Sum for slice {slice_id} is {total_sum}, which is not 1.")

        # Create a custom annotation array
        annot = np.vectorize(format_value)(div_matrix)

        # Plot heatmap with fixed scale from 0 to 1 and display formatted values
        sns.heatmap(div_matrix, ax=ax, cmap=cmap, vmin=0, vmax=1, cbar=True, 
                    annot=annot, fmt='', annot_kws={'size': 6})
        
        ax.set_title(f'Slice {slice_id}', fontsize=14)
        ax.set_xlabel('Effects', fontsize=12)
        ax.set_ylabel('Decisions', fontsize=12)
        
        # Adjust x-axis ticks with rotated labels
        ax.set_xticks(np.arange(len(all_effects)) + 0.5)
        ax.set_xticklabels(all_effects, rotation=45, ha='right', va='top', fontsize=6)
        
        # Adjust y-axis ticks
        ax.set_yticks(np.arange(len(sorted_slice_decisions)) + 0.5)
        ax.set_yticklabels(sorted_slice_decisions, rotation=0, fontsize=8)

        # Adjust layout
        plt.tight_layout()
        
        # Save the figure as PDF
        slice_output_path = output_path.replace('.pdf', f'_slice_{slice_id}.pdf')
        plt.savefig(slice_output_path, dpi=600, bbox_inches='tight')
        
        # Save the figure as LaTeX using tikzplotlib
        tex_output_path = slice_output_path.replace('.pdf', '.tex')
        tikzplotlib.save(tex_output_path)
        
        plt.close()

# Main execution
analysis_name = "Plots_for_Paper"
users = range(3, 7)
agents = ["embb-trf1", "embb-trf2", "urllc-trf1", "urllc-trf2"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0, 1, 2]
effects_list = create_effects_list()

for agent in agents:
    plot_path = create_plot_dir_for_results(analysis_name + f"/4_Heatmaps/Heatmap_of_Decision_Effect_per_slice_cleaned_decision/{agent}")
    
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    
    # Load all user scenarios data into one dataframe
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in users:
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data

    # Generate and save the heatmap
    output_file = os.path.join(plot_path, f"{agent}_heatmaps.pdf")
    plot_heatmaps_for_paper(agent_symbolic_data, agent, effects_list, kpis, slices, output_file)

print("All heatmaps have been generated and saved as PDF and LaTeX files.")

In [ ]:
plt.rcParams.update({'font.size': 8})  # Reduce default font size for better fit

def plot_heatmaps_for_paper(agent_symbolic_data, agent, effects_list, kpis, slices, output_path):
    # Concatenate all effects
    all_effects = [effect for kpi in kpis for effect in effects_list[kpi]]

    cmap = 'viridis'

    def format_value(val):
        if val >= 0.005:  # Changed threshold from 0.001 to 0.01
            return f'{val*100:.1f}%'  # Show as percentage with one decimal place
        return ''

    for slice_id in slices:
        slice_data = agent_symbolic_data[agent_symbolic_data['slice_id'] == slice_id]
        
        # Get unique decisions for this slice
        slice_decisions = slice_data['combined_decision'].unique()
        decision_counts = slice_data['combined_decision'].value_counts()
        sorted_slice_decisions = decision_counts.index.tolist()

        # Calculate the height based on the number of decisions in this slice
        decision_height = 0.3  # Height per decision in inches
        min_height = 5  # Minimum height of the plot
        calculated_height = max(min_height, len(sorted_slice_decisions) * decision_height)

        # Create a new figure for each slice
        fig, ax = plt.subplots(figsize=(20, calculated_height))
        fig.suptitle(f'Effect Probabilities Heatmap for Agent {agent} - Slice {slice_id}', fontsize=16)

        # Initialize the matrix for all KPIs and decisions in this slice
        div_matrix = np.zeros((len(sorted_slice_decisions), len(all_effects)))

        for i, decision in enumerate(sorted_slice_decisions):
            decision_data = slice_data[slice_data['combined_decision'] == decision]
            
            if not decision_data.empty:
                col_index = 0
                for kpi in kpis:
                    kpi_effects = effects_list[kpi]
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[i, col_index:col_index+len(kpi_effects)] = effect_counts.values
                    col_index += len(kpi_effects)

        # Normalize the matrix for this slice
        div_matrix = div_matrix / np.sum(div_matrix)

        # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
        total_sum = np.sum(div_matrix)
        if not np.isclose(total_sum, 1.0, atol=1e-6):
            print(f"Warning: Sum for slice {slice_id} is {total_sum}, which is not 1.")

        # Create a custom annotation array
        annot = np.vectorize(format_value)(div_matrix)

        # Plot heatmap with fixed scale from 0 to 1 and display formatted values
        sns.heatmap(div_matrix, ax=ax, cmap=cmap, vmin=0, vmax=1, cbar=True, 
                    annot=annot, fmt='', annot_kws={'size': 6})
        
        ax.set_title(f'Slice {slice_id}', fontsize=14)
        ax.set_xlabel('Effects', fontsize=12)
        ax.set_ylabel('Decisions', fontsize=12)
        
        # Adjust x-axis ticks with rotated labels
        ax.set_xticks(np.arange(len(all_effects)) + 0.5)
        ax.set_xticklabels(all_effects, rotation=45, ha='right', va='top', fontsize=6)
        
        # Adjust y-axis ticks
        ax.set_yticks(np.arange(len(sorted_slice_decisions)) + 0.5)
        ax.set_yticklabels(sorted_slice_decisions, rotation=0, fontsize=8)

        # Adjust layout
        plt.tight_layout()
        
        # Save the figure as PDF
        slice_output_path = output_path.replace('.pdf', f'_slice_{slice_id}.pdf')
        plt.savefig(slice_output_path, dpi=600, bbox_inches='tight')
        
        # Save the figure as LaTeX using tikzplotlib
        tex_output_path = slice_output_path.replace('.pdf', '.tex')
        tikzplotlib.save(tex_output_path)
        
        plt.close()

# Main execution
analysis_name = "Plots_for_Paper"
users = range(3, 7)
agents = ["embb-trf1", "embb-trf2", "urllc-trf1", "urllc-trf2"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0, 1, 2]
effects_list = create_effects_list()

for agent in agents:
    plot_path = create_plot_dir_for_results(analysis_name + f"/4_Heatmaps/Heatmap_of_Decision_Effect_per_slice_cleaned/{agent}")
    
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    
    # Load all user scenarios data into one dataframe
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in users:
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data

    # Generate and save the heatmap
    output_file = os.path.join(plot_path, f"{agent}_heatmaps.pdf")
    plot_heatmaps_for_paper(agent_symbolic_data, agent, effects_list, kpis, slices, output_file)

print("All heatmaps have been generated and saved as PDF and LaTeX files.")

### Heatmap of Decision Effect Per Agent for Slice Seperatly cleaned - Specific Request

This code will produce the specific density maps that we used in the paper. EMBB-TRF1 and URLLC-TRF1

In [ ]:
plt.rcParams.update({'font.size': 8})  # Reduce default font size for better fit

def plot_heatmaps_for_paper(agent_symbolic_data, agent, effects_list, kpis, slices, output_path):
    # Concatenate all effects
    all_effects = [effect for kpi in kpis for effect in effects_list[kpi]]

    cmap = 'viridis'

    def format_value(val):
        if val >= 0.005:  # Changed threshold from 0.001 to 0.01
            return f'{val*100:.1f}%'  # Show as percentage with one decimal place
        return ''

    for slice_id in slices:
        slice_data = agent_symbolic_data[agent_symbolic_data['slice_id'] == slice_id]
        
        # Get unique decisions for this slice
        slice_decisions = slice_data['combined_decision'].unique()
        decision_counts = slice_data['combined_decision'].value_counts()
        sorted_slice_decisions = decision_counts.index.tolist()

        # Calculate the height based on the number of decisions in this slice
        decision_height = 0.3  # Height per decision in inches
        min_height = 5  # Minimum height of the plot
        calculated_height = max(min_height, len(sorted_slice_decisions) * decision_height)

        # Create a new figure for each slice
        fig, ax = plt.subplots(figsize=(20, calculated_height))
        fig.suptitle(f'Effect Probabilities Heatmap for Agent {agent} - Slice {slice_id}', fontsize=16)

        # Initialize the matrix for all KPIs and decisions in this slice
        div_matrix = np.zeros((len(sorted_slice_decisions), len(all_effects)))

        for i, decision in enumerate(sorted_slice_decisions):
            decision_data = slice_data[slice_data['combined_decision'] == decision]
            
            if not decision_data.empty:
                col_index = 0
                for kpi in kpis:
                    kpi_effects = effects_list[kpi]
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[i, col_index:col_index+len(kpi_effects)] = effect_counts.values
                    col_index += len(kpi_effects)

        # Normalize the matrix for this slice
        div_matrix = div_matrix / np.sum(div_matrix)

        # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
        total_sum = np.sum(div_matrix)
        if not np.isclose(total_sum, 1.0, atol=1e-6):
            print(f"Warning: Sum for slice {slice_id} is {total_sum}, which is not 1.")

        # Create a custom annotation array
        annot = np.vectorize(format_value)(div_matrix)

        # Plot heatmap with fixed scale from 0 to 1 and display formatted values
        sns.heatmap(div_matrix, ax=ax, cmap=cmap, vmin=0, vmax=1, cbar=True, 
                    annot=annot, fmt='', annot_kws={'size': 6})
        
        ax.set_title(f'Slice {slice_id}', fontsize=14)
        ax.set_xlabel('Effects', fontsize=12)
        ax.set_ylabel('Decisions', fontsize=12)
        
        # Adjust x-axis ticks with rotated labels
        ax.set_xticks(np.arange(len(all_effects)) + 0.5)
        ax.set_xticklabels(all_effects, rotation=45, ha='right', va='top', fontsize=6)
        
        # Adjust y-axis ticks
        ax.set_yticks(np.arange(len(sorted_slice_decisions)) + 0.5)
        ax.set_yticklabels(sorted_slice_decisions, rotation=0, fontsize=8)

        # Adjust layout
        plt.tight_layout()
        
        # Save the figure as PDF
        slice_output_path = output_path.replace('.pdf', f'_slice_{slice_id}.pdf')
        plt.savefig(slice_output_path, dpi=600, bbox_inches='tight')
        
        # Save the figure as LaTeX using tikzplotlib
        tex_output_path = slice_output_path.replace('.pdf', '.tex')
        tikzplotlib.save(tex_output_path)
        
        plt.close()

# Main execution
analysis_name = "Plots_for_Paper"
users = range(3, 7)
agents = ["embb-trf1", "urllc-trf1"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0, 1, 2]
effects_list = create_effects_list()

for agent in agents:
    plot_path = create_plot_dir_for_results(analysis_name + f"/4_Heatmaps/Heatmap_of_Decision_Effect_per_slice_specific_scenario/{agent}")
    
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    
    # Load all user scenarios data into one dataframe
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in users:
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data

    # Generate and save the heatmap
    output_file = os.path.join(plot_path, f"{agent}_heatmaps.pdf")
    plot_heatmaps_for_paper(agent_symbolic_data, agent, effects_list, kpis, slices, output_file)

print("All heatmaps have been generated and saved as PDF and LaTeX files.")

##
------

## Specific Scenario

These codes will produce the specific scenario that we used in the paper.
Each version either apply a visualization affect for better readability or apply a more strict or easier threshold for showing or not showing the cells.

All the resutls are the same the difference is just in the way of presenting.

In [ ]:
plt.rcParams.update({'font.size': 8})  # Reduce default font size for better fit

def plot_heatmaps_for_paper(agent_symbolic_data, agent, effects_list, kpis, slices, output_path):
    cmap = 'viridis'

    def format_value(val):
        if val >= 0.005:  # Changed threshold from 0.001 to 0.01
            return f'{val*100:.1f}%'  # Show as percentage with one decimal place
        return ''

    for slice_id in slices:
        slice_data = agent_symbolic_data[agent_symbolic_data['slice_id'] == slice_id]
        
        # Get unique decisions for this slice
        slice_decisions = slice_data['combined_decision'].unique()
        decision_counts = slice_data['combined_decision'].value_counts()
        sorted_slice_decisions = decision_counts.index.tolist()

        # Initialize a list to store filtered decisions and their corresponding matrices for each KPI
        filtered_decisions_list = []
        filtered_div_matrices = []

        for kpi in kpis:
            kpi_effects = effects_list[kpi]
            # Initialize the matrix for the current KPI and decisions in this slice
            div_matrix = np.zeros((len(sorted_slice_decisions), len(kpi_effects)))

            for j, decision in enumerate(sorted_slice_decisions):
                decision_data = slice_data[slice_data['combined_decision'] == decision]
                
                if not decision_data.empty:
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[j] = effect_counts.values

            # Normalize the matrix for this slice
            div_matrix = div_matrix / np.sum(div_matrix)

            # Filter out rows where no cell has a value greater than 0.005
            row_mask = np.any(div_matrix > 0.005, axis=1)
            filtered_div_matrix = div_matrix[row_mask]
            filtered_decisions = np.array(sorted_slice_decisions)[row_mask]

            # Re-normalize the filtered matrix to ensure the sum is 1
            filtered_div_matrix = filtered_div_matrix / np.sum(filtered_div_matrix)

            # Save the filtered data for plotting
            filtered_decisions_list.append(filtered_decisions)
            filtered_div_matrices.append(filtered_div_matrix)

        # Determine the maximum number of rows to set the height of the plot dynamically
        max_rows = max(len(filtered_decisions) for filtered_decisions in filtered_decisions_list)
        plot_height = max(5, max_rows * 0.2)  # Set minimum height to 5 inches and 0.5 inches per row

        # Create a new figure for each slice with a 1x3 layout for the KPIs
        fig = plt.figure(figsize=(20, plot_height))
        gs = fig.add_gridspec(nrows=1, ncols=len(kpis) + 1, width_ratios=[1]*len(kpis) + [0.05])
        fig.suptitle(f'Effect Probabilities Heatmap for Agent {agent} - Slice {slice_id}', fontsize=16)

        axs = [fig.add_subplot(gs[0, i]) for i in range(len(kpis))]

        for i, (ax, kpi, filtered_div_matrix, filtered_decisions) in enumerate(zip(axs, kpis, filtered_div_matrices, filtered_decisions_list)):
            # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
            total_sum = np.sum(filtered_div_matrix)
            if not np.isclose(total_sum, 1.0, atol=1e-6):
                print(f"Warning: Sum for slice {slice_id}, KPI {kpi} is {total_sum}, which is not 1.")

            # Create a custom annotation array
            annot = np.vectorize(format_value)(filtered_div_matrix)

            # Plot heatmap with fixed scale from 0 to 1 and display formatted values
            sns.heatmap(filtered_div_matrix, ax=ax, cmap=cmap, vmin=0, vmax=1, cbar=False, 
                        annot=annot, fmt='', annot_kws={'size': 6})
            
            ax.set_title(f'{kpi}', fontsize=14)
            ax.set_xlabel('Effects', fontsize=12)

            # Only set y-axis labels for the first subplot
            if i == 0:
                ax.set_ylabel('Decisions', fontsize=12)
                ax.set_yticks(np.arange(len(filtered_decisions)) + 0.5)
                ax.set_yticklabels(filtered_decisions, rotation=0, fontsize=8)
            else:
                ax.set_yticklabels([])
                ax.set_ylabel('')

            # Adjust x-axis ticks with rotated labels
            ax.set_xticks(np.arange(len(kpi_effects)) + 0.5)
            ax.set_xticklabels(kpi_effects, rotation=45, ha='right', va='top', fontsize=6)

        # Add a single colorbar to the right of all subplots
        cbar_ax = fig.add_subplot(gs[0, -1])
        fig.colorbar(axs[0].collections[0], cbar_ax, orientation='vertical')

        # Adjust layout
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        
        # Save the figure as PDF
        slice_output_path = output_path.replace('.pdf', f'_slice_{slice_id}.pdf')
        plt.savefig(slice_output_path, dpi=600, bbox_inches='tight')
        
        # Save the figure as LaTeX using tikzplotlib
        # tex_output_path = slice_output_path.replace('.pdf', '.tex')
        # tikzplotlib_fix_ncols
        # tikzplotlib.save(tex_output_path)
        
        plt.close()

# Main execution
analysis_name = "Plots_for_Paper"
users = range(3, 7)
agents = ["embb-trf1", "urllc-trf1"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0, 1, 2]
effects_list = create_effects_list()

for agent in agents:
    plot_path = create_plot_dir_for_results(analysis_name + f"/4_Heatmaps/Heatmap_of_Decision_Effect_per_slice_specific_scenario/{agent}")
    
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    
    # Load all user scenarios data into one dataframe
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in users:
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data

    # Generate and save the heatmap
    output_file = os.path.join(plot_path, f"{agent}_heatmaps.pdf")
    plot_heatmaps_for_paper(agent_symbolic_data, agent, effects_list, kpis, slices, output_file)

print("All heatmaps have been generated and saved as PDF and LaTeX files.")

### V3

#### Removed unsued rows

In [ ]:
def plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_path):
    cmap = 'viridis'

    def format_value(val):
        if val >= 0.01:
            return f'{val*100:.1f}%'
        return ''

    # Initialize a 2x3 layout for the combined plot (2 agents x 3 KPIs)
    fig, axs = plt.subplots(2, 3, figsize=(20, 10))
    fig.suptitle('Combined Heatmaps for Agents and Slices', fontsize=16)

    for row, (agent, slice_id) in enumerate(agent_data_dict.items()):
        slice_data = slice_id['data']
        agent_name = slice_id['agent']
        
        # Get unique decisions for this slice
        slice_decisions = slice_data['combined_decision'].unique()
        decision_counts = slice_data['combined_decision'].value_counts()
        sorted_slice_decisions = decision_counts.index.tolist()

        # Initialize a list to store filtered decisions and their corresponding matrices for each KPI
        filtered_decisions_list = []
        filtered_div_matrices = []

        for kpi in kpis:
            kpi_effects = effects_list[kpi]
            # Initialize the matrix for the current KPI and decisions in this slice
            div_matrix = np.zeros((len(sorted_slice_decisions), len(kpi_effects)))

            for j, decision in enumerate(sorted_slice_decisions):
                decision_data = slice_data[slice_data['combined_decision'] == decision]
                
                if not decision_data.empty:
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[j] = effect_counts.values

            # Normalize the matrix for this slice
            div_matrix = div_matrix / np.sum(div_matrix)

            # Filter out rows where no cell has a value greater than 0.008
            row_mask = np.any(div_matrix > 0.008, axis=1)
            filtered_div_matrix = div_matrix[row_mask]
            filtered_decisions = np.array(sorted_slice_decisions)[row_mask]

            # Re-normalize the filtered matrix to ensure the sum is 1
            filtered_div_matrix = filtered_div_matrix / np.sum(filtered_div_matrix)

            # Save the filtered data for plotting
            filtered_decisions_list.append(filtered_decisions)
            filtered_div_matrices.append(filtered_div_matrix)

        for col, (ax, kpi, filtered_div_matrix, filtered_decisions) in enumerate(zip(axs[row], kpis, filtered_div_matrices, filtered_decisions_list)):
            # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
            total_sum = np.sum(filtered_div_matrix)
            if not np.isclose(total_sum, 1.0, atol=1e-6):
                print(f"Warning: Sum for slice {slice_id}, KPI {kpi} is {total_sum}, which is not 1.")

            # Create a custom annotation array
            annot = np.vectorize(format_value)(filtered_div_matrix)

            # Plot heatmap with fixed scale from 0 to 1 and display formatted values
            sns.heatmap(filtered_div_matrix, ax=ax, cmap=cmap, vmin=0, vmax=1, cbar=False, 
                        annot=annot, fmt='', annot_kws={'size': 6})
            
            ax.set_title(f'{agent_name} - {kpi}', fontsize=14)
            if row == 1:
                ax.set_xlabel('Effects', fontsize=12)

            # Only set y-axis labels for the first subplot in each row
            if col == 0:
                ax.set_ylabel('Decisions', fontsize=12)
                ax.set_yticks(np.arange(len(filtered_decisions)) + 0.5)
                ax.set_yticklabels(filtered_decisions, rotation=0, fontsize=8)
            else:
                ax.set_yticklabels([])
                ax.set_ylabel('')

            # Adjust x-axis ticks with rotated labels
            ax.set_xticks(np.arange(len(kpi_effects)) + 0.5)
            ax.set_xticklabels(kpi_effects, rotation=45, ha='right', va='top', fontsize=6)

        # Remove x-axis labels for the first row
        if row == 0:
            for col in range(3):
                axs[row, col].set_xticklabels([])

    # Add a single color bar to the right of the plot
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    norm = plt.Normalize(vmin=0, vmax=1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig.colorbar(sm, cax=cbar_ax)

    # Adjust layout
    plt.subplots_adjust(wspace=0.001, hspace=0.01)  # Reduce these values to decrease space between subplots
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    
    # Save the figure as PDF
    plt.savefig(output_path, dpi=600, bbox_inches='tight')
    
    # Save the figure as LaTeX using tikzplotlib
    tex_output_path = output_path.replace('.pdf', '.tex')
    tikzplotlib.save(tex_output_path)
    
    plt.close()

# Main execution
agents = ["embb-trf1", "urllc-trf1"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0]
effects_list = create_effects_list()

agent_data_dict = {}

for agent in agents:
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in range(3, 7):
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data

    agent_data_dict[agent] = {'data': agent_symbolic_data[agent_symbolic_data['slice_id'] == 0], 'agent': agent}

# Generate and save the heatmap
plot_path = create_plot_dir_for_results(analysis_name + f"/4_Heatmaps/Heatmap_of_Decision_Effect_per_slice_specific_scenario/V3/")
output_file = os.path.join(plot_path, f"combined_heatmaps.pdf")

plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_file)

print("Combined heatmap has been generated and saved as PDF and LaTeX files.")

### V3.1

In [ ]:
def plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_path):
    cmap = 'viridis'

    # Define the threshold values for each agent
    agent_thresholds = {
        "embb-trf1": 0.01,  # 1 percent
        "urllc-trf1": 0.02  # 2 percent
    }

    def format_value(val, threshold):
        if np.isnan(val):
            return ''
        if val >= threshold:
            return f'{round(val * 100):d}'
        return ''

    # Initialize a 2x3 layout for the combined plot (2 agents x 3 KPIs)
    fig, axs = plt.subplots(2, 3, figsize=(20, 10))
    fig.suptitle('Combined Heatmaps for Agents and Slices', fontsize=16)

    # Initialize lists to store data for both rows
    row_filtered_decisions_list = []
    row_filtered_div_matrices_list = []
    row_filtered_effects_list = []

    for row, (agent, slice_id) in enumerate(agent_data_dict.items()):
        slice_data = slice_id['data']
        agent_name = slice_id['agent']
        threshold = agent_thresholds[agent_name]
        
        # Get unique decisions for this slice
        slice_decisions = slice_data['combined_decision'].unique()
        decision_counts = slice_data['combined_decision'].value_counts()
        sorted_slice_decisions = decision_counts.index.tolist()

        # Initialize a mask for significant decisions across all KPIs
        significant_decisions_mask = np.zeros(len(sorted_slice_decisions), dtype=bool)

        # First pass: determine significant decisions across all KPIs
        for kpi in kpis:
            kpi_effects = effects_list[kpi]
            div_matrix = np.zeros((len(sorted_slice_decisions), len(kpi_effects)))

            for j, decision in enumerate(sorted_slice_decisions):
                decision_data = slice_data[slice_data['combined_decision'] == decision]
                
                if not decision_data.empty:
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[j] = effect_counts.values

            div_matrix = div_matrix / np.sum(div_matrix)
            significant_decisions_mask |= np.any(div_matrix > threshold, axis=1)

        # Filter decisions using the unified mask
        filtered_decisions = np.array(sorted_slice_decisions)[significant_decisions_mask]

        # Second pass: create filtered matrices for each KPI
        filtered_div_matrices = []
        filtered_effects_list = []

        for kpi in kpis:
            kpi_effects = effects_list[kpi]
            div_matrix = np.zeros((len(sorted_slice_decisions), len(kpi_effects)))

            for j, decision in enumerate(sorted_slice_decisions):
                decision_data = slice_data[slice_data['combined_decision'] == decision]
                
                if not decision_data.empty:
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[j] = effect_counts.values

            div_matrix = div_matrix / np.sum(div_matrix)
            filtered_div_matrix = div_matrix[significant_decisions_mask]

            # Re-normalize the filtered matrix
            filtered_div_matrix = filtered_div_matrix / np.sum(filtered_div_matrix)

            filtered_div_matrices.append(filtered_div_matrix)
            filtered_effects_list.append(kpi_effects)

        # Store the filtered data for this row
        row_filtered_decisions_list.append([filtered_decisions] * len(kpis))
        row_filtered_div_matrices_list.append(filtered_div_matrices)
        row_filtered_effects_list.append(filtered_effects_list)

    # Determine columns to keep across both rows for each KPI
    final_filtered_effects_list = []
    for kpi_idx in range(len(kpis)):
        combined_mask = np.zeros(len(effects_list[kpis[kpi_idx]]), dtype=bool)
        for row_filtered_div_matrices in row_filtered_div_matrices_list:
            combined_mask |= np.any(row_filtered_div_matrices[kpi_idx] > 0.008, axis=0)
        final_filtered_effects = np.array(effects_list[kpis[kpi_idx]])[combined_mask]
        final_filtered_effects_list.append(final_filtered_effects)

        # Re-filter the matrices based on the combined mask
        for row in range(2):
            row_filtered_div_matrices_list[row][kpi_idx] = row_filtered_div_matrices_list[row][kpi_idx][:, combined_mask]

    # Plot the heatmaps
    for row in range(2):
        agent_name = agents[row]
        threshold = agent_thresholds[agent_name]
        for col, (ax, kpi, filtered_div_matrix, filtered_decisions, final_filtered_effects) in enumerate(zip(
                axs[row], kpis, row_filtered_div_matrices_list[row], row_filtered_decisions_list[row], final_filtered_effects_list)):
            # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
            total_sum = np.sum(filtered_div_matrix)
            if not np.isclose(total_sum, 1.0, atol=1e-6):
                print(f"Warning: Sum for agent {agents[row]}, KPI {kpi} is {total_sum}, which is not 1.")

            # Mask values lower than the threshold
            display_matrix = np.where(filtered_div_matrix >= threshold, filtered_div_matrix, np.nan)

            # Re-normalize the display values to ensure the sum is 1
            display_matrix = display_matrix / np.nansum(display_matrix)

            # Create a custom annotation array
            annot = np.vectorize(format_value)(display_matrix, threshold)

            # Plot heatmap with fixed scale from 0 to 1 and display formatted values
            sns.heatmap(display_matrix, ax=ax, cmap=cmap, vmin=0, vmax=1, cbar=False, 
                        annot=annot, fmt='', annot_kws={'size': 6}, mask=np.isnan(display_matrix))
            
            ax.set_title(f'{agents[row]} - {kpi}', fontsize=14)
            if row == 1:
                ax.set_xlabel('Effects', fontsize=12)

            # Only set y-axis labels for the first subplot in each row
            if col == 0:
                ax.set_ylabel(f'Decisions ({agents[row]})', fontsize=12)
                ax.set_yticks(np.arange(len(filtered_decisions)) + 0.5)
                ax.set_yticklabels(filtered_decisions, rotation=0, fontsize=8)
            else:
                ax.set_yticklabels([])
                ax.set_ylabel('')

            # Adjust x-axis ticks with rotated labels
            ax.set_xticks(np.arange(len(final_filtered_effects)) + 0.5)
            ax.set_xticklabels(final_filtered_effects, rotation=45, ha='right', va='top', fontsize=6)

        # Remove x-axis labels for the first row
        if row == 0:
            for col in range(3):
                axs[row, col].set_xticklabels([])

    # Add a single color bar to the right of the plot
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    norm = plt.Normalize(vmin=0, vmax=1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig.colorbar(sm, cax=cbar_ax)

    # Adjust layout
    plt.subplots_adjust(wspace=0.001, hspace=0.01)
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    
    # Save the figure as PDF
    plt.savefig(output_path, dpi=600, bbox_inches='tight')
    
    # Save the figure as LaTeX using tikzplotlib
    tex_output_path = output_path.replace('.pdf', '.tex')
    tikzplotlib.save(tex_output_path)
    
    plt.close()

# The rest of your code remains the same
analysis_name = "Plots_for_Paper"
agents = ["embb-trf1", "urllc-trf1"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0]
effects_list = create_effects_list()

agent_data_dict = {}

for agent in agents:
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in range(3, 7):
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data

    agent_data_dict[agent] = {'data': agent_symbolic_data[agent_symbolic_data['slice_id'] == 0], 'agent': agent}

# Generate and save the heatmap
plot_path = create_plot_dir_for_results(analysis_name + f"/4_Heatmaps/Heatmap_of_Decision_Effect_per_slice_specific_scenario/V3.2_masked_No_DN_Filter_White/")
output_file = os.path.join(plot_path, f"combined_heatmaps.pdf")

plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_file)

print("Combined heatmap has been generated and saved as PDF and LaTeX files.")

### masked

In [ ]:
def plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_path):
    cmap = 'viridis'

    def format_value(val):
        if val >= 0.01:
            return f'{val*100:.1f}%'
        return ''

    # Initialize a 2x3 layout for the combined plot (2 agents x 3 KPIs)
    fig, axs = plt.subplots(2, 3, figsize=(20, 10))
    fig.suptitle('Combined Heatmaps for Agents and Slices', fontsize=16)

    # Initialize lists to store data for both rows
    row_filtered_decisions_list = []
    row_filtered_div_matrices_list = []
    row_filtered_effects_list = []

    for row, (agent, slice_id) in enumerate(agent_data_dict.items()):
        slice_data = slice_id['data']
        agent_name = slice_id['agent']
        
        # Get unique decisions for this slice
        slice_decisions = slice_data['combined_decision'].unique()
        decision_counts = slice_data['combined_decision'].value_counts()
        sorted_slice_decisions = decision_counts.index.tolist()

        # Initialize a list to store filtered decisions and their corresponding matrices for each KPI
        filtered_decisions_list = []
        filtered_div_matrices = []
        filtered_effects_list = []

        for kpi in kpis:
            kpi_effects = effects_list[kpi]
            # Initialize the matrix for the current KPI and decisions in this slice
            div_matrix = np.zeros((len(sorted_slice_decisions), len(kpi_effects)))

            for j, decision in enumerate(sorted_slice_decisions):
                decision_data = slice_data[slice_data['combined_decision'] == decision]
                
                if not decision_data.empty:
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[j] = effect_counts.values

            # Normalize the matrix for this slice
            div_matrix = div_matrix / np.sum(div_matrix)

            # Filter out rows where no cell has a value greater than 0.008
            row_mask = np.any(div_matrix > 0.008, axis=1)
            filtered_div_matrix = div_matrix[row_mask]
            filtered_decisions = np.array(sorted_slice_decisions)[row_mask]

            # Append the filtered matrices and decisions to the lists
            filtered_decisions_list.append(filtered_decisions)
            filtered_div_matrices.append(filtered_div_matrix)
            filtered_effects_list.append(kpi_effects)

        # Store the filtered data for this row
        row_filtered_decisions_list.append(filtered_decisions_list)
        row_filtered_div_matrices_list.append(filtered_div_matrices)
        row_filtered_effects_list.append(filtered_effects_list)

    # Determine columns to keep across both rows for each KPI
    final_filtered_effects_list = []
    for kpi_idx in range(len(kpis)):
        combined_mask = np.zeros(len(effects_list[kpis[kpi_idx]]), dtype=bool)
        for row_filtered_div_matrices in row_filtered_div_matrices_list:
            combined_mask |= np.any(row_filtered_div_matrices[kpi_idx] > 0.008, axis=0)
        final_filtered_effects = np.array(effects_list[kpis[kpi_idx]])[combined_mask]
        final_filtered_effects_list.append(final_filtered_effects)

        # Re-filter the matrices and decisions based on the combined mask
        for row in range(2):
            row_filtered_div_matrices_list[row][kpi_idx] = row_filtered_div_matrices_list[row][kpi_idx][:, combined_mask]

    # Plot the heatmaps
    for row in range(2):
        for col, (ax, kpi, filtered_div_matrix, filtered_decisions, final_filtered_effects) in enumerate(zip(
                axs[row], kpis, row_filtered_div_matrices_list[row], row_filtered_decisions_list[row], final_filtered_effects_list)):
            # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
            total_sum = np.sum(filtered_div_matrix)
            if not np.isclose(total_sum, 1.0, atol=1e-6):
                print(f"Warning: Sum for agent {agents[row]}, KPI {kpi} is {total_sum}, which is not 1.")

            # Create a custom annotation array
            annot = np.vectorize(format_value)(filtered_div_matrix)

            # Plot heatmap with fixed scale from 0 to 1 and display formatted values
            sns.heatmap(filtered_div_matrix, ax=ax, cmap=cmap, vmin=0, vmax=1, cbar=False, 
                        annot=annot, fmt='', annot_kws={'size': 6})
            
            ax.set_title(f'{agents[row]} - {kpi}', fontsize=14)
            if row == 1:
                ax.set_xlabel('Effects', fontsize=12)

            # Only set y-axis labels for the first subplot in each row
            if col == 0:
                ax.set_ylabel('Decisions', fontsize=12)
                ax.set_yticks(np.arange(len(filtered_decisions)) + 0.5)
                ax.set_yticklabels(filtered_decisions, rotation=0, fontsize=8)
            else:
                ax.set_yticklabels([])
                ax.set_ylabel('')

            # Adjust x-axis ticks with rotated labels
            ax.set_xticks(np.arange(len(final_filtered_effects)) + 0.5)
            ax.set_xticklabels(final_filtered_effects, rotation=45, ha='right', va='top', fontsize=6)

        # Remove x-axis labels for the first row
        if row == 0:
            for col in range(3):
                axs[row, col].set_xticklabels([])

    # Add a single color bar to the right of the plot
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    norm = plt.Normalize(vmin=0, vmax=1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig.colorbar(sm, cax=cbar_ax)

    # Adjust layout
    plt.subplots_adjust(wspace=0.001, hspace=0.01)  # Reduce these values to decrease space between subplots
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    
    # Save the figure as PDF
    plt.savefig(output_path, dpi=600, bbox_inches='tight')
    
    # Save the figure as LaTeX using tikzplotlib
    tex_output_path = output_path.replace('.pdf', '.tex')
    tikzplotlib.save(tex_output_path)
    
    plt.close()


analysis_name = "Plots_for_Paper"
agents = ["embb-trf1", "urllc-trf1"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0]
effects_list = create_effects_list()

agent_data_dict = {}

for agent in agents:
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in range(3, 7):
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data

    agent_data_dict[agent] = {'data': agent_symbolic_data[agent_symbolic_data['slice_id'] == 0], 'agent': agent}

# Generate and save the heatmap
plot_path = create_plot_dir_for_results(analysis_name + f"/4_Heatmaps/Heatmap_of_Decision_Effect_per_slice_specific_scenario/V3_masked/")
output_file = os.path.join(plot_path, f"combined_heatmaps.pdf")

plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_file)

print("Combined heatmap has been generated and saved as PDF and LaTeX files.")

### V3.2 masked

#### Without decimal points

In [ ]:
def plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_path):
    cmap = 'viridis'

    # Define the threshold values for each agent
    agent_thresholds = {
        "embb-trf1": 0.01,  # 1 percent
        "urllc-trf1": 0.02  # 2 percent
    }

    def format_value(val, threshold):
        if val >= threshold:
            return f'{round(val * 100):d}'
        return ''

    # Initialize a 2x3 layout for the combined plot (2 agents x 3 KPIs)
    fig, axs = plt.subplots(2, 3, figsize=(20, 10))
    fig.suptitle('Combined Heatmaps for Agents and Slices', fontsize=16)

    # Initialize lists to store data for both rows
    row_filtered_decisions_list = []
    row_filtered_div_matrices_list = []
    row_filtered_effects_list = []

    for row, (agent, slice_id) in enumerate(agent_data_dict.items()):
        slice_data = slice_id['data']
        agent_name = slice_id['agent']
        threshold = agent_thresholds[agent_name]
        
        # Get unique decisions for this slice
        slice_decisions = slice_data['combined_decision'].unique()
        decision_counts = slice_data['combined_decision'].value_counts()
        sorted_slice_decisions = decision_counts.index.tolist()

        # Initialize a list to store filtered decisions and their corresponding matrices for each KPI
        filtered_decisions_list = []
        filtered_div_matrices = []
        filtered_effects_list = []

        for kpi in kpis:
            kpi_effects = effects_list[kpi]
            # Initialize the matrix for the current KPI and decisions in this slice
            div_matrix = np.zeros((len(sorted_slice_decisions), len(kpi_effects)))

            for j, decision in enumerate(sorted_slice_decisions):
                decision_data = slice_data[slice_data['combined_decision'] == decision]
                
                if not decision_data.empty:
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[j] = effect_counts.values

            # Normalize the matrix for this slice
            div_matrix = div_matrix / np.sum(div_matrix)

            # Filter out rows where no cell has a value greater than the threshold
            row_mask = np.any(div_matrix > threshold, axis=1)
            filtered_div_matrix = div_matrix[row_mask]
            filtered_decisions = np.array(sorted_slice_decisions)[row_mask]

            # Re-normalize the filtered matrix to ensure the sum is 1
            filtered_div_matrix = filtered_div_matrix / np.sum(filtered_div_matrix)

            # Append the filtered matrices and decisions to the lists
            filtered_decisions_list.append(filtered_decisions)
            filtered_div_matrices.append(filtered_div_matrix)
            filtered_effects_list.append(kpi_effects)

        # Store the filtered data for this row
        row_filtered_decisions_list.append(filtered_decisions_list)
        row_filtered_div_matrices_list.append(filtered_div_matrices)
        row_filtered_effects_list.append(filtered_effects_list)

    # Determine columns to keep across both rows for each KPI
    final_filtered_effects_list = []
    for kpi_idx in range(len(kpis)):
        combined_mask = np.zeros(len(effects_list[kpis[kpi_idx]]), dtype=bool)
        for row_filtered_div_matrices in row_filtered_div_matrices_list:
            combined_mask |= np.any(row_filtered_div_matrices[kpi_idx] > 0.008, axis=0)
        final_filtered_effects = np.array(effects_list[kpis[kpi_idx]])[combined_mask]
        final_filtered_effects_list.append(final_filtered_effects)

        # Re-filter the matrices and decisions based on the combined mask
        for row in range(2):
            row_filtered_div_matrices_list[row][kpi_idx] = row_filtered_div_matrices_list[row][kpi_idx][:, combined_mask]

    # Plot the heatmaps
    for row in range(2):
        agent_name = agents[row]
        threshold = agent_thresholds[agent_name]
        for col, (ax, kpi, filtered_div_matrix, filtered_decisions, final_filtered_effects) in enumerate(zip(
                axs[row], kpis, row_filtered_div_matrices_list[row], row_filtered_decisions_list[row], final_filtered_effects_list)):
            # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
            total_sum = np.sum(filtered_div_matrix)
            if not np.isclose(total_sum, 1.0, atol=1e-6):
                print(f"Warning: Sum for agent {agents[row]}, KPI {kpi} is {total_sum}, which is not 1.")

            # Create a custom annotation array
            annot = np.vectorize(format_value)(filtered_div_matrix, threshold)

            # Re-normalize the display values to ensure the sum is 1
            display_matrix = np.where(filtered_div_matrix >= threshold, filtered_div_matrix, 0)
            display_matrix = display_matrix / np.sum(display_matrix)

            annot = np.vectorize(format_value)(display_matrix, threshold)

            # Plot heatmap with fixed scale from 0 to 1 and display formatted values
            sns.heatmap(display_matrix, ax=ax, cmap=cmap, vmin=0, vmax=1, cbar=False, 
                        annot=annot, fmt='', annot_kws={'size': 6})
            
            ax.set_title(f'{agents[row]} - {kpi}', fontsize=14)
            if row == 1:
                ax.set_xlabel('Effects', fontsize=12)

            # Only set y-axis labels for the first subplot in each row
            if col == 0:
                ax.set_ylabel(f'Decisions ({agents[row]})', fontsize=12)
                ax.set_yticks(np.arange(len(filtered_decisions)) + 0.5)
                ax.set_yticklabels(filtered_decisions, rotation=0, fontsize=8)
            else:
                ax.set_yticklabels([])
                ax.set_ylabel('')

            # Adjust x-axis ticks with rotated labels
            ax.set_xticks(np.arange(len(final_filtered_effects)) + 0.5)
            ax.set_xticklabels(final_filtered_effects, rotation=45, ha='right', va='top', fontsize=6)

        # Remove x-axis labels for the first row
        if row == 0:
            for col in range(3):
                axs[row, col].set_xticklabels([])

    # Add a single color bar to the right of the plot
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    norm = plt.Normalize(vmin=0, vmax=1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig.colorbar(sm, cax=cbar_ax)

    # Adjust layout
    plt.subplots_adjust(wspace=0.001, hspace=0.01)  # Reduce these values to decrease space between subplots
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    
    # Save the figure as PDF
    plt.savefig(output_path, dpi=600, bbox_inches='tight')
    
    # Save the figure as LaTeX using tikzplotlib
    tex_output_path = output_path.replace('.pdf', '.tex')
    tikzplotlib.save(tex_output_path)
    
    plt.close()


analysis_name = "Plots_for_Paper"
agents = ["embb-trf1", "urllc-trf1"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0]
effects_list = create_effects_list()

agent_data_dict = {}

for agent in agents:
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in range(3, 7):
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data

    agent_data_dict[agent] = {'data': agent_symbolic_data[agent_symbolic_data['slice_id'] == 0], 'agent': agent}

# Generate and save the heatmap
plot_path = create_plot_dir_for_results(analysis_name + f"/4_Heatmaps/Heatmap_of_Decision_Effect_per_slice_specific_scenario/V3.2_masked_No_DN/")
output_file = os.path.join(plot_path, f"combined_heatmaps.pdf")

plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_file)

print("Combined heatmap has been generated and saved as PDF and LaTeX files.")

#### Witout Decimal Poitns by filtering out empty cells

In [ ]:
def plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_path):
    cmap = 'viridis'

    # Define the threshold values for each agent
    agent_thresholds = {
        "embb-trf1": 0.01,  # 1 percent
        "urllc-trf1": 0.02  # 2 percent
    }

    def format_value(val, threshold):
        if np.isnan(val):
            return ''
        if val >= threshold:
            return f'{round(val * 100):d}'
        return ''

    # Initialize a 2x3 layout for the combined plot (2 agents x 3 KPIs)
    fig, axs = plt.subplots(2, 3, figsize=(20, 10))
    fig.suptitle('Combined Heatmaps for Agents and Slices', fontsize=16)

    # Initialize lists to store data for both rows
    row_filtered_decisions_list = []
    row_filtered_div_matrices_list = []
    row_filtered_effects_list = []

    for row, (agent, slice_id) in enumerate(agent_data_dict.items()):
        slice_data = slice_id['data']
        agent_name = slice_id['agent']
        threshold = agent_thresholds[agent_name]
        
        # Get unique decisions for this slice
        slice_decisions = slice_data['combined_decision'].unique()
        decision_counts = slice_data['combined_decision'].value_counts()
        sorted_slice_decisions = decision_counts.index.tolist()

        # Initialize a mask for significant decisions across all KPIs
        significant_decisions_mask = np.zeros(len(sorted_slice_decisions), dtype=bool)

        # First pass: determine significant decisions across all KPIs
        for kpi in kpis:
            kpi_effects = effects_list[kpi]
            div_matrix = np.zeros((len(sorted_slice_decisions), len(kpi_effects)))

            for j, decision in enumerate(sorted_slice_decisions):
                decision_data = slice_data[slice_data['combined_decision'] == decision]
                
                if not decision_data.empty:
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[j] = effect_counts.values

            div_matrix = div_matrix / np.sum(div_matrix)
            significant_decisions_mask |= np.any(div_matrix > threshold, axis=1)

        # Filter decisions using the unified mask
        filtered_decisions = np.array(sorted_slice_decisions)[significant_decisions_mask]

        # Second pass: create filtered matrices for each KPI
        filtered_div_matrices = []
        filtered_effects_list = []

        for kpi in kpis:
            kpi_effects = effects_list[kpi]
            div_matrix = np.zeros((len(sorted_slice_decisions), len(kpi_effects)))

            for j, decision in enumerate(sorted_slice_decisions):
                decision_data = slice_data[slice_data['combined_decision'] == decision]
                
                if not decision_data.empty:
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[j] = effect_counts.values

            div_matrix = div_matrix / np.sum(div_matrix)
            filtered_div_matrix = div_matrix[significant_decisions_mask]

            # Re-normalize the filtered matrix
            filtered_div_matrix = filtered_div_matrix / np.sum(filtered_div_matrix)

            filtered_div_matrices.append(filtered_div_matrix)
            filtered_effects_list.append(kpi_effects)

        # Store the filtered data for this row
        row_filtered_decisions_list.append([filtered_decisions] * len(kpis))
        row_filtered_div_matrices_list.append(filtered_div_matrices)
        row_filtered_effects_list.append(filtered_effects_list)

    # Determine columns to keep across both rows for each KPI
    final_filtered_effects_list = []
    for kpi_idx in range(len(kpis)):
        combined_mask = np.zeros(len(effects_list[kpis[kpi_idx]]), dtype=bool)
        for row_filtered_div_matrices in row_filtered_div_matrices_list:
            combined_mask |= np.any(row_filtered_div_matrices[kpi_idx] > 0.008, axis=0)
        final_filtered_effects = np.array(effects_list[kpis[kpi_idx]])[combined_mask]
        final_filtered_effects_list.append(final_filtered_effects)

        # Re-filter the matrices based on the combined mask
        for row in range(2):
            row_filtered_div_matrices_list[row][kpi_idx] = row_filtered_div_matrices_list[row][kpi_idx][:, combined_mask]

    # Plot the heatmaps
    for row in range(2):
        agent_name = agents[row]
        threshold = agent_thresholds[agent_name]
        for col, (ax, kpi, filtered_div_matrix, filtered_decisions, final_filtered_effects) in enumerate(zip(
                axs[row], kpis, row_filtered_div_matrices_list[row], row_filtered_decisions_list[row], final_filtered_effects_list)):
            # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
            total_sum = np.sum(filtered_div_matrix)
            if not np.isclose(total_sum, 1.0, atol=1e-6):
                print(f"Warning: Sum for agent {agents[row]}, KPI {kpi} is {total_sum}, which is not 1.")

            # Mask values lower than the threshold
            display_matrix = np.where(filtered_div_matrix >= threshold, filtered_div_matrix, np.nan)

            # Re-normalize the display values to ensure the sum is 1
            display_matrix = display_matrix / np.nansum(display_matrix)

            # Create a custom annotation array
            annot = np.vectorize(format_value)(display_matrix, threshold)

            # Plot heatmap with fixed scale from 0 to 1 and display formatted values
            sns.heatmap(display_matrix, ax=ax, cmap=cmap, vmin=0, vmax=1, cbar=False, 
                        annot=annot, fmt='', annot_kws={'size': 6}, mask=np.isnan(display_matrix))
            
            ax.set_title(f'{agents[row]} - {kpi}', fontsize=14)
            if row == 1:
                ax.set_xlabel('Effects', fontsize=12)

            # Only set y-axis labels for the first subplot in each row
            if col == 0:
                ax.set_ylabel(f'Decisions ({agents[row]})', fontsize=12)
                ax.set_yticks(np.arange(len(filtered_decisions)) + 0.5)
                ax.set_yticklabels(filtered_decisions, rotation=0, fontsize=8)
            else:
                ax.set_yticklabels([])
                ax.set_ylabel('')

            # Adjust x-axis ticks with rotated labels
            ax.set_xticks(np.arange(len(final_filtered_effects)) + 0.5)
            ax.set_xticklabels(final_filtered_effects, rotation=45, ha='right', va='top', fontsize=6)

        # Remove x-axis labels for the first row
        if row == 0:
            for col in range(3):
                axs[row, col].set_xticklabels([])

    # Add a single color bar to the right of the plot
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    norm = plt.Normalize(vmin=0, vmax=1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig.colorbar(sm, cax=cbar_ax)

    # Adjust layout
    plt.subplots_adjust(wspace=0.001, hspace=0.01)
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    
    # Save the figure as PDF
    plt.savefig(output_path, dpi=600, bbox_inches='tight')
    
    # Save the figure as LaTeX using tikzplotlib
    tex_output_path = output_path.replace('.pdf', '.tex')
    tikzplotlib.save(tex_output_path)
    
    plt.close()

# The rest of your code remains the same
analysis_name = "Plots_for_Paper"
agents = ["embb-trf1", "urllc-trf1"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0]
effects_list = create_effects_list()

agent_data_dict = {}

for agent in agents:
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in range(3, 7):
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data

    agent_data_dict[agent] = {'data': agent_symbolic_data[agent_symbolic_data['slice_id'] == 0], 'agent': agent}

# Generate and save the heatmap
plot_path = create_plot_dir_for_results(analysis_name + f"/4_Heatmaps/Heatmap_of_Decision_Effect_per_slice_specific_scenario/V3.2_masked_No_DN_Filter_White/")
output_file = os.path.join(plot_path, f"combined_heatmaps.pdf")

plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_file)

print("Combined heatmap has been generated and saved as PDF and LaTeX files.")

#### With one Decimal Points

In [ ]:
def plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_path):
    cmap = 'viridis'

    # Define the threshold values for each agent
    agent_thresholds = {
        "embb-trf1": 0.005,  # 1 percent
        "urllc-trf1": 0.02  # 2 percent
    }

    def format_value(val, threshold):
        if val >= threshold:
            return f'{val * 100:.1f}'
        return ''

    # Initialize a 2x3 layout for the combined plot (2 agents x 3 KPIs)
    fig, axs = plt.subplots(2, 3, figsize=(20, 10))
    fig.suptitle('Combined Heatmaps for Agents and Slices', fontsize=16)

    # Initialize lists to store data for both rows
    row_filtered_decisions_list = []
    row_filtered_div_matrices_list = []
    row_filtered_effects_list = []

    for row, (agent, slice_id) in enumerate(agent_data_dict.items()):
        slice_data = slice_id['data']
        agent_name = slice_id['agent']
        threshold = agent_thresholds[agent_name]
        
        # Get unique decisions for this slice
        slice_decisions = slice_data['combined_decision'].unique()
        decision_counts = slice_data['combined_decision'].value_counts()
        sorted_slice_decisions = decision_counts.index.tolist()

        # Initialize a list to store filtered decisions and their corresponding matrices for each KPI
        filtered_decisions_list = []
        filtered_div_matrices = []
        filtered_effects_list = []

        for kpi in kpis:
            kpi_effects = effects_list[kpi]
            # Initialize the matrix for the current KPI and decisions in this slice
            div_matrix = np.zeros((len(sorted_slice_decisions), len(kpi_effects)))

            for j, decision in enumerate(sorted_slice_decisions):
                decision_data = slice_data[slice_data['combined_decision'] == decision]
                
                if not decision_data.empty:
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[j] = effect_counts.values

            # Normalize the matrix for this slice
            div_matrix = div_matrix / np.sum(div_matrix)

            # Filter out rows where no cell has a value greater than the threshold
            row_mask = np.any(div_matrix > threshold, axis=1)
            filtered_div_matrix = div_matrix[row_mask]
            filtered_decisions = np.array(sorted_slice_decisions)[row_mask]

            # Re-normalize the filtered matrix to ensure the sum is 1
            filtered_div_matrix = filtered_div_matrix / np.sum(filtered_div_matrix)

            # Append the filtered matrices and decisions to the lists
            filtered_decisions_list.append(filtered_decisions)
            filtered_div_matrices.append(filtered_div_matrix)
            filtered_effects_list.append(kpi_effects)

        # Store the filtered data for this row
        row_filtered_decisions_list.append(filtered_decisions_list)
        row_filtered_div_matrices_list.append(filtered_div_matrices)
        row_filtered_effects_list.append(filtered_effects_list)

    # Determine columns to keep across both rows for each KPI
    final_filtered_effects_list = []
    for kpi_idx in range(len(kpis)):
        combined_mask = np.zeros(len(effects_list[kpis[kpi_idx]]), dtype=bool)
        for row_filtered_div_matrices in row_filtered_div_matrices_list:
            combined_mask |= np.any(row_filtered_div_matrices[kpi_idx] > 0.008, axis=0)
        final_filtered_effects = np.array(effects_list[kpis[kpi_idx]])[combined_mask]
        final_filtered_effects_list.append(final_filtered_effects)

        # Re-filter the matrices and decisions based on the combined mask
        for row in range(2):
            row_filtered_div_matrices_list[row][kpi_idx] = row_filtered_div_matrices_list[row][kpi_idx][:, combined_mask]

    # Plot the heatmaps
    for row in range(2):
        agent_name = agents[row]
        threshold = agent_thresholds[agent_name]
        for col, (ax, kpi, filtered_div_matrix, filtered_decisions, final_filtered_effects) in enumerate(zip(
                axs[row], kpis, row_filtered_div_matrices_list[row], row_filtered_decisions_list[row], final_filtered_effects_list)):
            # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
            total_sum = np.sum(filtered_div_matrix)
            if not np.isclose(total_sum, 1.0, atol=1e-6):
                print(f"Warning: Sum for agent {agents[row]}, KPI {kpi} is {total_sum}, which is not 1.")

            # Create a custom annotation array
            annot = np.vectorize(format_value)(filtered_div_matrix, threshold)

            # Re-normalize the display values to ensure the sum is 1
            display_matrix = np.where(filtered_div_matrix >= threshold, filtered_div_matrix, 0)
            display_matrix = display_matrix / np.sum(display_matrix)

            annot = np.vectorize(format_value)(display_matrix, threshold)

            # Plot heatmap with fixed scale from 0 to 1 and display formatted values
            sns.heatmap(display_matrix, ax=ax, cmap=cmap, vmin=0, vmax=1, cbar=False, 
                        annot=annot, fmt='', annot_kws={'size': 6})
            
            ax.set_title(f'{agents[row]} - {kpi}', fontsize=14)
            if row == 1:
                ax.set_xlabel('Effects', fontsize=12)

            # Only set y-axis labels for the first subplot in each row
            if col == 0:
                ax.set_ylabel(f'Decisions ({agents[row]})', fontsize=12)
                ax.set_yticks(np.arange(len(filtered_decisions)) + 0.5)
                ax.set_yticklabels(filtered_decisions, rotation=0, fontsize=8)
            else:
                ax.set_yticklabels([])
                ax.set_ylabel('')

            # Adjust x-axis ticks with rotated labels
            ax.set_xticks(np.arange(len(final_filtered_effects)) + 0.5)
            ax.set_xticklabels(final_filtered_effects, rotation=45, ha='right', va='top', fontsize=6)

        # Remove x-axis labels for the first row
        if row == 0:
            for col in range(3):
                axs[row, col].set_xticklabels([])

    # Add a single color bar to the right of the plot
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    norm = plt.Normalize(vmin=0, vmax=1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig.colorbar(sm, cax=cbar_ax)

    # Adjust layout
    plt.subplots_adjust(wspace=0.001, hspace=0.01)  # Reduce these values to decrease space between subplots
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    
    # Save the figure as PDF
    plt.savefig(output_path, dpi=600, bbox_inches='tight')
    
    # Save the figure as LaTeX using tikzplotlib
    tex_output_path = output_path.replace('.pdf', '.tex')
    tikzplotlib.save(tex_output_path)
    
    plt.close()


analysis_name = "Plots_for_Paper"
agents = ["embb-trf1", "urllc-trf1"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0]
effects_list = create_effects_list()

agent_data_dict = {}

for agent in agents:
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in range(3, 7):
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data

    agent_data_dict[agent] = {'data': agent_symbolic_data[agent_symbolic_data['slice_id'] == 0], 'agent': agent}

# Generate and save the heatmap
plot_path = create_plot_dir_for_results(analysis_name + f"/4_Heatmaps/Heatmap_of_Decision_Effect_per_slice_specific_scenario/V3.2_masked_With_DN/")
output_file = os.path.join(plot_path, f"combined_heatmaps.pdf")

plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_file)

print("Combined heatmap has been generated and saved as PDF and LaTeX files.")

#### With decimal points by filtering out empty cells 

In [ ]:
def plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_path):
    cmap = 'viridis'

    # Define the threshold values for each agent
    agent_thresholds = {
        "embb-trf1": 0.005,  # 1 percent
        "urllc-trf1": 0.02  # 2 percent
    }

    def format_value(val, threshold):
        if val >= threshold:
            return f'{val * 100:.1f}'
        return ''

    # Initialize a 2x3 layout for the combined plot (2 agents x 3 KPIs)
    fig, axs = plt.subplots(2, 3, figsize=(20, 10))
    fig.suptitle('Combined Heatmaps for Agents and Slices', fontsize=16)

    # Initialize lists to store data for both rows
    row_filtered_decisions_list = []
    row_filtered_div_matrices_list = []
    row_filtered_effects_list = []

    for row, (agent, slice_id) in enumerate(agent_data_dict.items()):
        slice_data = slice_id['data']
        agent_name = slice_id['agent']
        threshold = agent_thresholds[agent_name]
        
        # Get unique decisions for this slice
        slice_decisions = slice_data['combined_decision'].unique()
        decision_counts = slice_data['combined_decision'].value_counts()
        sorted_slice_decisions = decision_counts.index.tolist()

        # Initialize a list to store filtered decisions and their corresponding matrices for each KPI
        filtered_decisions_list = []
        filtered_div_matrices = []
        filtered_effects_list = []

        for kpi in kpis:
            kpi_effects = effects_list[kpi]
            # Initialize the matrix for the current KPI and decisions in this slice
            div_matrix = np.zeros((len(sorted_slice_decisions), len(kpi_effects)))

            for j, decision in enumerate(sorted_slice_decisions):
                decision_data = slice_data[slice_data['combined_decision'] == decision]
                
                if not decision_data.empty:
                    effect_counts = decision_data[kpi].value_counts().reindex(kpi_effects, fill_value=0)
                    div_matrix[j] = effect_counts.values

            # Normalize the matrix for this slice
            div_matrix = div_matrix / np.sum(div_matrix)

            # Filter out rows where no cell has a value greater than the threshold
            row_mask = np.any(div_matrix > threshold, axis=1)
            filtered_div_matrix = div_matrix[row_mask]
            filtered_decisions = np.array(sorted_slice_decisions)[row_mask]

            # Re-normalize the filtered matrix to ensure the sum is 1
            filtered_div_matrix = filtered_div_matrix / np.sum(filtered_div_matrix)

            # Append the filtered matrices and decisions to the lists
            filtered_decisions_list.append(filtered_decisions)
            filtered_div_matrices.append(filtered_div_matrix)
            filtered_effects_list.append(kpi_effects)

        # Store the filtered data for this row
        row_filtered_decisions_list.append(filtered_decisions_list)
        row_filtered_div_matrices_list.append(filtered_div_matrices)
        row_filtered_effects_list.append(filtered_effects_list)

    # Determine columns to keep across both rows for each KPI
    final_filtered_effects_list = []
    for kpi_idx in range(len(kpis)):
        combined_mask = np.zeros(len(effects_list[kpis[kpi_idx]]), dtype=bool)
        for row_filtered_div_matrices in row_filtered_div_matrices_list:
            combined_mask |= np.any(row_filtered_div_matrices[kpi_idx] > 0.008, axis=0)
        final_filtered_effects = np.array(effects_list[kpis[kpi_idx]])[combined_mask]
        final_filtered_effects_list.append(final_filtered_effects)

        # Re-filter the matrices and decisions based on the combined mask
        for row in range(2):
            row_filtered_div_matrices_list[row][kpi_idx] = row_filtered_div_matrices_list[row][kpi_idx][:, combined_mask]

    # Plot the heatmaps
    for row in range(2):
        agent_name = agents[row]
        threshold = agent_thresholds[agent_name]
        for col, (ax, kpi, filtered_div_matrix, filtered_decisions, final_filtered_effects) in enumerate(zip(
                axs[row], kpis, row_filtered_div_matrices_list[row], row_filtered_decisions_list[row], final_filtered_effects_list)):
            # Verify that the sum is 1 (or very close to 1 due to floating-point precision)
            total_sum = np.sum(filtered_div_matrix)
            if not np.isclose(total_sum, 1.0, atol=1e-6):
                print(f"Warning: Sum for agent {agents[row]}, KPI {kpi} is {total_sum}, which is not 1.")

            # Create a custom annotation array
            annot = np.vectorize(format_value)(filtered_div_matrix, threshold)

            # Re-normalize the display values to ensure the sum is 1
            display_matrix = np.where(filtered_div_matrix >= threshold, filtered_div_matrix, 0)
            display_matrix = display_matrix / np.sum(display_matrix)

            # Update annotation array after re-normalization
            annot = np.vectorize(format_value)(display_matrix, threshold)

            # Create a mask to hide values below the threshold
            mask = display_matrix == 0

            # Plot heatmap with fixed scale from 0 to 1 and display formatted values
            sns.heatmap(display_matrix, ax=ax, cmap=cmap, vmin=0, vmax=1, cbar=False, 
                        annot=annot, fmt='', annot_kws={'size': 6}, mask=mask)
            
            ax.set_title(f'{agents[row]} - {kpi}', fontsize=14)
            if row == 1:
                ax.set_xlabel('Effects', fontsize=12)

            # Only set y-axis labels for the first subplot in each row
            if col == 0:
                ax.set_ylabel(f'Decisions ({agents[row]})', fontsize=12)
                ax.set_yticks(np.arange(len(filtered_decisions)) + 0.5)
                ax.set_yticklabels(filtered_decisions, rotation=0, fontsize=8)
            else:
                ax.set_yticklabels([])
                ax.set_ylabel('')

            # Adjust x-axis ticks with rotated labels
            ax.set_xticks(np.arange(len(final_filtered_effects)) + 0.5)
            ax.set_xticklabels(final_filtered_effects, rotation=45, ha='right', va='top', fontsize=6)

        # Remove x-axis labels for the first row
        if row == 0:
            for col in range(3):
                axs[row, col].set_xticklabels([])

    # Add a single color bar to the right of the plot
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    norm = plt.Normalize(vmin=0, vmax=1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig.colorbar(sm, cax=cbar_ax)

    # Adjust layout
    plt.subplots_adjust(wspace=0.001, hspace=0.01)  # Reduce these values to decrease space between subplots
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    
    # Save the figure as PDF
    plt.savefig(output_path, dpi=600, bbox_inches='tight')
    
    # Save the figure as LaTeX using tikzplotlib
    tex_output_path = output_path.replace('.pdf', '.tex')
    tikzplotlib.save(tex_output_path)
    
    plt.close()


analysis_name = "Plots_for_Paper"
agents = ["embb-trf1", "urllc-trf1"]
kpis = ['tx_brate', 'tx_pckts', 'dl_buffer']
slices = [0]
effects_list = create_effects_list()

agent_data_dict = {}

for agent in agents:
    agent_info = AGENT_EXPERIMENT_INFO[agent]
    agent_symbolic_data = pd.DataFrame()
    for number_of_user in range(3, 7):
        kpi_data, decision_data = handle_data(agent_info, number_of_user)
        symbolic_effects, _ = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, number_of_user)
        symbolic_effects['user_num'] = number_of_user
        symbolic_effects['combined_decision'] = symbolic_effects.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)
        agent_symbolic_data = pd.concat([agent_symbolic_data, symbolic_effects], axis=0)
        del symbolic_effects, kpi_data, decision_data

    agent_data_dict[agent] = {'data': agent_symbolic_data[agent_symbolic_data['slice_id'] == 0], 'agent': agent}

# Generate and save the heatmap
plot_path = create_plot_dir_for_results(analysis_name + f"/4_Heatmaps/Heatmap_of_Decision_Effect_per_slice_specific_scenario/V3.2_masked_With_DN_Filter_white/")
output_file = os.path.join(plot_path, f"combined_heatmaps.pdf")

plot_combined_heatmaps_for_paper(agent_data_dict, agents, effects_list, kpis, slices, output_file)

print("Combined heatmap has been generated and saved as PDF and LaTeX files.")

##
------